In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
import re

In [63]:
df = pd.read_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/KBM_data/users_w_no_KBM.csv')
pckg = pd.read_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/KBM_data/KBM_package_225K.csv')
pckg['pckg'] = 1
pckg = pckg[['user_id','pckg']]
df = pd.merge(df, pckg, on = 'user_id', how = 'left')
df = df.loc[pd.isnull(df['pckg'])]
df = df[['user_id', 'License', 'first_name', 'patronymic_name', 'last_name','birthday', 'rus_ctzn']]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
df['birthday_dt'] = pd.to_datetime(df['birthday'], errors = 'coerce')
df['license_f'] = df['License'].str.replace(' ','')
df['birthday_error'] = ((pd.to_datetime('today')-df['birthday_dt']).astype('timedelta64[Y]') < 19.0)|\
                        ((pd.to_datetime('today')-df['birthday_dt']).astype('timedelta64[Y]') > 80.0)|\
                        (pd.isnull(df['birthday_dt']))
for i in ['first_name', 'patronymic_name', 'last_name']:
    if i == 'patronymic_name':
        df.loc[(df[i].str.isalpha() == False)|\
               (df[i] == 'Нет')|\
               (df[i] == 'нет')|\
               (df[i] == 'Отсутствует')|\
               (df[i] == 'отсутствует')|\
               (df[i] == 'В')|\
               (df[i] == 'Процессе')|\
               (df[i] == 'Регистрации')|\
               (df[i].str.isnumeric() == True)] = np.nan
        df[i+'_nick'] = df[i].str.contains('Мойка')|\
                        df[i].str.contains('Каршайн')|\
                        df[i].str.contains('Карсервис')|\
                        df[i].str.contains('Омывайка')|\
                        df[i].str.contains('Нафта')|\
                        df[i].str.contains('Пройл')|\
                        df[i].str.contains('Пролив')
        df[i+'_latin'] = df[i].str.contains(pat = '[a-zA-Z]', regex = True)
    else:
        df[i+'_NA'] = ((pd.isnull(df[i]))|\
                       (df[i].str.isalpha() == False)|\
                       (df[i] == 'Нет')|\
                       (df[i] == 'нет')|\
                       (df[i] == 'Отсутствует')|\
                       (df[i] == 'отсутствует')|\
                       (df[i] == 'В')|\
                       (df[i] == 'Процессе')|\
                       (df[i] == 'Регистрации'))
        df[i+'_nick'] = df[i].str.contains('Мойка')|\
                        df[i].str.contains('Каршайн')|\
                        df[i].str.contains('Карсервис')|\
                        df[i].str.contains('Омывайка')|\
                        df[i].str.contains('Нафта')|\
                        df[i].str.contains('Пройл')|\
                        df[i].str.contains('Пролив')
        df[i+'_latin'] = df[i].str.contains(pat = '[a-zA-Z]', regex = True)
df['FIO_NA'] = (df['first_name_NA'] == True)|(df['last_name_NA'] == True)
df['FIO_nick'] = (df['first_name_nick'] == True)|(df['patronymic_name_nick'] == True)|(df['last_name_nick'] == True)
df['FIO_latin'] = (df['first_name_latin'] == True)|(df['patronymic_name_latin'] == True)|(df['last_name_latin'] == True)
df['license_error'] = (df['license_f'].str.len()!=10.0)|(pd.isnull(df.license_f))
df['any_error'] = (df.license_error == True)|(df.birthday_error == True)|(df.FIO_NA == True)|(df.FIO_nick == True)|(df.FIO_latin == True)
df_clean = df.loc[(df.birthday_error == False)&(df.FIO_NA == False)&(df.FIO_nick == False)&(df.FIO_latin == False)]
output = df_clean[['user_id','first_name', 'patronymic_name', 'last_name','birthday_dt', 'license_f', 'any_error']]
output1 = output.loc[output.index <= 550000]
output2 = output.loc[output.index > 550000]
output1.to_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/KBM_data/users_w_no_KBM_cleaned1.csv')
output2.to_csv('C:/Users/sgulbin/Work/Scoring (Accidents)/2020-03Test_v2/KBM_data/users_w_no_KBM_cleaned2.csv')
df_rus = df.loc[df.rus_ctzn == 1.0]
for i in ['birthday_error', 'license_error', 'FIO_NA', 'FIO_nick', 'FIO_latin', 'any_error']:
    print(i,df_clean[i].value_counts())

birthday_error 0.0    804463
Name: birthday_error, dtype: int64
license_error False    752276
True      52187
Name: license_error, dtype: int64
FIO_NA False    804463
Name: FIO_NA, dtype: int64
FIO_nick False    804463
Name: FIO_nick, dtype: int64
FIO_latin False    804463
Name: FIO_latin, dtype: int64
any_error False    752276
True      52187
Name: any_error, dtype: int64
